In [1]:
API_KEY = 'XXX'


In [2]:
# To install the Python SDK, use this CLI command:
# pip install google-generativeai

import google.generativeai as genai
from google.generativeai import GenerativeModel

API_KEY = 'XXX'
genai.configure(api_key=API_KEY)

model = GenerativeModel("gemini-2.0-flash")

response = model.generate_content("The opposite of hot is")

print(response.text) #  The opposite of hot is cold.
        

C:\Users\user\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [ ]:
#https://ai.google.dev/gemini-api/docs/structured-output
from google import genai
from pydantic import BaseModel

class Recipe(BaseModel):
    recipe_name: str
    ingredients: list[str]

client = genai.Client(api_key=API_KEY)
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="List a few popular cookie recipes, and include the amounts of ingredients.",
    config={
        "response_mime_type": "application/json",
        "response_schema": list[Recipe],
    },
)
# Use the response as a JSON string.
print(response.text)

# Use instantiated objects.
my_recipes: list[Recipe] = response.parsed

In [ ]:
#https://ai.google.dev/gemini-api/docs/grounding?lang=python
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

client = genai.Client(api_key=API_KEY)
model_id = "gemini-2.0-flash"

google_search_tool = Tool(
    google_search = GoogleSearch()
)

response = client.models.generate_content(
    model=model_id,
    contents="When is the next total solar eclipse in the United States?",
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

for each in response.candidates[0].content.parts:
    print("***")
    print(each.text)
# Example response:
# The next total solar eclipse visible in the contiguous United States will be on ...

# To get grounding metadata as web content.
print("###")
print(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

In [ ]:
from google import genai
from google.genai.types import (
    GenerateContentConfig,
    GoogleSearch,
    HttpOptions,
    Tool,
)

API_KEY = 'XXX' # <-- Replace this placeholder

client = genai.Client(api_key=API_KEY, http_options=HttpOptions(api_version="v1"))

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="When is the next total solar eclipse in the United States?",
    config=GenerateContentConfig(
        tools=[
            # Use Google Search Tool
            Tool(google_search=GoogleSearch())
        ],
    ),
)

print(response.text)
# Example response:
# 'The next total solar eclipse in the United States will occur on ...'

In [ ]:
# Make sure you have installed/upgraded the library using:
# pip install --upgrade google-generativeai

import google.generativeai as genai
import os
# Import necessary tool classes - verify based on your library version
# The exact import might be genai.Tool or from google.generativeai import Tool
from google.generativeai import GenerationConfig, Tool
# If Tool.from_Google Search_web doesn't exist, check genai.tools.search or documentation
#from google.generativeai.tools import search
from google.generativeai import GoogleSearch

# --- Configuration ---
# Replace with your actual API key.
# It's highly recommended to load this from an environment variable
# rather than hardcoding it, especially in shared code or notebooks.
# Example: API_KEY = os.environ.get("GOOGLE_API_KEY")
API_KEY = 'XXX' # <-- Replace this placeholder

# Configure the genai library with your API key
try:
    genai.configure(api_key=API_KEY)
except Exception as e:
    print(f"Error configuring API key: {e}")
    print("Please ensure you have a valid API key set.")
    exit() # Stop execution if configuration fails

# --- System Instruction & Prompt Text ---
system_instruction_text = """
You are a helpful assistant specifically tasked with finding publicly licensed 
images from Wikimedia Commons for tourist attractions.
Your output *must* be a valid JSON object containing a single key "wikimedia_urls" 
with a list of exactly 3 image URLs.
Each URL must point to a real image file on Wikimedia Commons (no "file not found" errors).
All images must be published with a license suitable for free commercial use, 
such as CC BY-SA 4.0.
Select 3 images: one overall view/exterior, one interior view or significant detail, 
and another distinct detail.
Do not include any text before or after the JSON object.
Focus your search on Wikimedia Commons files for the given point of interest.
""" # Added explicit instruction to focus search on Wikimedia

prompt_text = """
Your task is providing URLs to images of tourist attractions useful for a travel guide.

#Take into account following information:
{'id': 'POI-02', 'latitude': 48.3408, 'longitude': 15.2419, 'name': 'Burgruine Lichtenegg (Castle Ruin)'}

#Instructions:
Select 3 pictures from Wikimedia, one should give an overall impression of the 
point of interest, the others should be on distinct interesting details. 
Alternatively provide images of one exterior view, one interior view, and one detail.
##Constraints: It is important, that the image really exist, i.e. no message on 
website like "No file by this name exists.". Additionally, the images need to be 
published with a license suitable for free commercial use, such as CC BY-SA 4.0. 

##Output: Return the answer as a valid JSON in the same format as in the following example. 
Take care that the JSON is valid.

##Example:
{
  "wikimedia_urls": [
    "https://commons.wikimedia.org/wiki/File:Kath._Pfarrkirche_hl._Martin_in_Sankt_Martin_-_Altar.jpg",
    "https://commons.wikimedia.org/wiki/File:Kath._Pfarrkirche_hl._Martin_in_Sankt_Martin_2017.jpg",
    "https://commons.wikimedia.org/wiki/File:2012.05.05_-_St._Martin_-_Pfarrkirche_hl._Martin_-_07.jpg"
  ]
}
"""

# --- Define and Add the Search Tool ---
try:
    # Define the tool(s) - Google Search is the standard for finding web info
    # This gives the model access to perform search queries
    # The exact method name might vary slightly based on library version.
    # Common approach:
    #available_tools = [
    #    Tool.from_google_search_web()
        # If this gives an error, try checking the documentation for Tool or genai.tools.search
        # e.g., from google.generativeai.tools import search
    #    available_tools = [search.GoogleSearch()] # Example if using the submodule
    #]


    # --- Get the model instance with system instruction AND tools ---
    # Pass system_instruction and tools when you get the model
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash-preview-04-17", # Use the model name
        system_instruction=system_instruction_text, # Pass system instruction here
        #tools=available_tools # <-- Add the tools parameter here!
        tools=Tool(google_search=GoogleSearch())
    )

    print("Model initialized with search tool.")

except Exception as e:
    print(f"Error initializing model with tools: {e}")
    print("Ensure your google-generativeai library version supports tools and the correct tool definition.")
    print("Check the library's documentation for 'Tools' or 'Function Calling'.")
    exit()


# --- Generate Content ---
try:
    print("Generating content...")
    response = model.generate_content(
        contents=[
            {"role": "user", "parts": [{"text": prompt_text}]},
        ],
        generation_config=GenerationConfig( # Use GenerationConfig object explicitly
             temperature=0.6, # Use = for keyword args
             response_mime_type="application/json"
        ),
        # tool_config is generally not needed here if tools are added to the model,
        # as adding tools implicitly allows the model to use them.
        # tool_config={"function_calling_config": "ANY"} # Optional: Can sometimes help
    )

    # --- Process Response ---
    # Check if the response was blocked or if candidates are missing
    if not response or not response.candidates:
        print("Response was empty or blocked.")
        # More detailed feedback might be in response.prompt_feedback or candidates
        if hasattr(response, 'prompt_feedback') and response.prompt_feedback and response.prompt_feedback.block_reason:
             print(f"Block reason: {response.prompt_feedback.block_reason}")
        if hasattr(response, 'candidates') and response.candidates and hasattr(response.candidates[0], 'finish_reason'):
             print(f"Finish reason: {response.candidates[0].finish_reason}")

        # Check if the model returned a function call instead of text
        if hasattr(response.candidates[0].content, 'parts'):
             for part in response.candidates[0].content.parts:
                 if part.function_call:
                     print("\nModel requested a function call (e.g., search) instead of returning text.")
                     print("This means the model decided to use the tool.")
                     print("Function call details:", part.function_call)
                     print("\nTo get the final answer, you would typically execute this tool call and send the results back to the model.")
                     print("However, for a simple search tool added to the model, the model often executes it internally and returns the final text response.")
                     print("If you are still not getting text, you might need to implement a multi-turn function calling flow.")

        exit()

    # Access the response text
    # The model should use the tool internally and return JSON if successful
    print("Response received successfully:")
    print(response.text)

    # Optional: Add JSON parsing here as shown before
    # import json
    # try:
    #     json_data = json.loads(response.text)
    #     print("\nParsed JSON:")
    #     print(json.dumps(json_data, indent=2))
    # except json.JSONDecodeError:
    #     print("\nFailed to parse response as JSON.")
    #     print(response.text) # Print raw text if parsing fails


except Exception as e:
    print(f"Error during content generation: {e}")
